Consider the 1d shallow water equations 

\begin{align} 
    h_{t}+{(hu)}_x &= f^h \;, \\ 
    (hu)_{t}+{\left(hu^2+\frac{1}{2}gh^2\right)}_x & = f^{hu}\;,
\end{align}

with a manufactured solution 

\begin{align*} 
    h(x,t) = \frac{1+x}{1+t}\;, \quad  u(x,t)  = \frac{1+x^2}{0.5+t} \;.
\end{align*}

on the spatial domain $[0,1]$. This code solves the problem upto final time t = 0.5 and save the data.

In [ ]:
from clawpack.pyclaw.solution import Solution
import numpy as np
import matplotlib.pyplot as plt
from clawpack import visclaw
from clawpack.visclaw.ianimate import ianimate
from nodepy import rk
import os, sys
import scipy.io
import scipy.integrate as integrate
from Num_Sol_1D_SWE import setup

In [ ]:
def butcher_coeff(s,p,q,sch_no):
# Coefficients of Runge-Kutta method
#------------------------------#
    # Method 1: (3,3,1)
    if s == 3 and p == 3 and q == 1 and sch_no == 1:
        ssp33 = rk.loadRKM('SSP33')
        a = ssp33.A.astype('float64')
        b = ssp33.b.astype('float64')
        c = ssp33.c.astype('float64')
    # Method 2 (5,3,3)
    elif s == 5 and p == 3 and q == 3 and sch_no == 2:
#------------------------------#
        # Method 2: (5,3,3)
        a = np.array([
                    [0, 0, 0, 0, 0],
                    [3/11, 0, 0, 0, 0],
                    [285645/493487, 103950/493487, 0, 0, 0],
                    [3075805/5314896, 1353275/5314896, 0, 0, 0],
                    [196687/177710, -129383023/426077496, 48013/42120, -2268/2405, 0]
                    ])
        b = np.array([(5626/4725),(-25289/13608),(569297/340200),(324/175),(-13/7)])
        c = np.array([0,(3/11),(15/19),(5/6),1])
#------------------------------#
    # Method 3: (4,4,1)
    elif s == 4 and p == 4 and q == 1 and sch_no == 3:        
        rk4 = rk.loadRKM('RK44')
        a = rk4.A.astype('float64')
        b = rk4.b.astype('float64')
        c = rk4.c.astype('float64')
#------------------------------#
    # Method 4: (7,4,4)
    elif s == 7 and p == 4 and q == 4 and sch_no == 4:
        a = np.array([
                    [0, 0, 0, 0, 0, 0, 0],
                    [13/15, 0, 0, 0, 0, 0, 0],
                    [12603078031712033723970154667732315345979717833/24160502835995108267237194998715463575206403200,
                    1048907966089364624562691286403757878851144981/72481508507985324801711584996146390725619209600,
                    0, 0, 0, 0, 0],
                    [599677/612720, 1/185, 1/69, 0, 0, 0, 0],
                    [424559865415888618629601372734144061495213187/3221400378132681102298292666495395143360853760,
                    2833374238559988231687781191318699943620201/644280075626536220459658533299079028672170752,
                    10939005/8358742409, 0, 0, 0, 0],
                    [-57910884850960710216615685584299594389734738212701891/143200771200355662082106164157818074407670810332226816,
                    1058112267371143700865117923757427517682611842306739/12613539432155680079771009278149778574768931480040704,
                    -16209173194776291095/101264438213449707492,-32817015/650336938, 19/34, 0, 0],
                    [265884392436244759286808856348403413175819281494905197368751/270149059788838487928804328430790305367090530076692853841920,
                    -79382446353850270662474337268761271598625606923168650489267/55522863756255233270341476827744466906189590810235664261120,
                    60065272366553932534499312161/31525704659895735897804153776,181308146225/261301556177, -88/41, 51/64, 0]
                    ])
        b = np.array([(-124664924851382288077728137/37822531451473304797511250),(60663184710162550730781989/9804158397668360944968750),(-276006970775403888708156064/46485365179396605438524625),(-265040017375280744373155776/74664519359574416777071875),(100421079686366171194956352/16321974107859175324078125),(57748716265592512/317670660091351875),(3694532608/2928669975)])
        c = np.array([0,(13/15),(193/360),(719/720),(11/80),(1/36),(193/240)])
#------------------------------#
    # Method 5: (7,5,1)
    elif s == 7 and p == 5 and q == 1 and sch_no == 5:
        dp5 = rk.loadRKM('DP5')
        a = dp5.A.astype('float64')
        b = dp5.b.astype('float64')
        c = dp5.c.astype('float64')
#------------------------------#
    # Method 6: (9,5,5)
    elif s == 9 and p == 5 and q == 5 and sch_no == 6: 
        a = np.array([
                    [0,0,0,0,0,0,0,0,0],
                    [(1/19),0,0,0,0,0,0,0,0],
                    [(1/6),0,0,0,0,0,0,0,0],
                    [(5/16),0,0,0,0,0,0,0,0],
                    [(1/2),0,0,0,0,0,0,0,0],
                    [(11/16),0,0,0,0,0,0,0,0],
                    [(11448031/2850816),(-67411795275/16590798848),(51073011/43237376),(-23353/64148),(583825/8077312),(-1/116),0,0,0],
                    [(30521441823091/1986340257792),(-745932230071621375/35792226257928192),(42324456085/5966757888),(775674925/6453417096),(-38065236125/28020473856),(18388001255/24775053336),(-25/138),0,0],
                    [(544015925591990906117739018863/21097279127167116142731264000),(-51819957177912933732533469147783191/1292529408768612025127952939417600),(15141148893501140337719772533/769541606770966638202880000),(-22062343808701233885761491/5740046662014404900523000),(-180818957612953115541011736739/146721986657116762265358336000),(18393837528018836258241002593/22366927394951953576613895000),(-14372715851/701966192290),(-3316780581/34682124125),0]
                    ])
        b = np.array([(201919428075343316424206867/7205146638186855485778750),(-979811820279525173317561445351/23232888464237446713644747250),(-659616477161155066954978/262813990730721440278125),(10343523856053877739219144704/232857239079584284108576875),(-2224588357354685208355760476/50108519801935858605643125),(704220346724742597999572733952/31288349276326419946994221875),(-13778944/1751475),(92889088/11941875),(-714103988224/149255126145)])
        c = np.array([0,(1/19),(1/6),(5/16),(1/2),(11/16),(5/6),(16/17),1])
#------------------------------#
    return a,b,c

In [ ]:
def fun_h(x,t):
    return (1+x)/(1+t) # manufactured solution such that there is no spatial error 

def fun_u(x,t):
    return (1+x**2)/(0.5+t) # manufactured solution such that there is no spatial error 

# momentum function
def fun_hu(x,t):
    return fun_h(x,t)*fun_u(x,t)

def fun_h_cell_avg(x,t):
    dx = x[1] - x[0]; sol = np.zeros((len(x)))
    for i in range(len(x)):
        I1 = integrate.quad(lambda x: fun_h(x,t), x[i]-dx/2,x[i]+dx/2)
        sol[i] = I1[0]/dx
    return sol

def fun_hu_cell_avg(x,t):
    dx = x[1] - x[0]; sol = np.zeros((len(x)))
    for i in range(len(x)):
        I1 = integrate.quad(lambda x: fun_hu(x,t), x[i]-dx/2,x[i]+dx/2)
        sol[i] = I1[0]/dx
    return sol

def compute_err(N, s, p, q, sch_no, solver_type, tfinal, cfl):
    errs_h = []; errs_hu = [];
    for idx in range(len(N)):
        print(N[idx])
        a,b,c = butcher_coeff(s, p, q, sch_no)
        claw = setup(mx=N[idx],a=a,b=b,c=c,solver_type=solver_type,tfinal=tfinal, cfl = cfl) 
        claw.run()
        frame = claw.frames[-1]; xc = frame.grid.x.centers; t = tfinal
        num_h = frame.q[0]; num_hu = frame.q[1]
        true_h = fun_h_cell_avg(xc,t); true_hu = fun_hu_cell_avg(xc,t); 
        err_h = np.max(np.abs(true_h-num_h)); err_hu = np.max(np.abs(true_hu-num_hu))
        errs_h.append(err_h); errs_hu.append(err_hu)
    return errs_h, errs_hu

## Convergence test

In [ ]:
N = np.ceil(5*10**np.linspace(1,2.5,8))[:-1]; 
solver_type='sharpclaw'; tfinal=0.5; cfl = 0.8; dts = cfl*(1./N)/2; 
S = [3,5,4,7,7,9];P =[3,3,4,4,5,5]; Q =[1,3,1,4,1,5]; SCH_NO = [1,2,3,4,5,6]


# uncomment here to rerun the code
Err_u = np.zeros((len(S),len(N))); Err_hu = np.zeros((len(S),len(N)))

for i in range(len(S)):
    print("(Method = (%d,%d,%d))"%(S[i], P[i], Q[i]))
    errs_h, errs_hu = compute_err(N, S[i], P[i], Q[i], SCH_NO[i], solver_type, tfinal, cfl)
    Err_u[i,:] = errs_h; Err_hu[i,:] = errs_hu;

#### Saving data

In [ ]:
folder_name = "SWE_1D_ConvgData/"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
#Convergence plot data
np.save("./%s/SWE_1D_dts_CFL_%1.1f_T_%1.1f.npy"%(folder_name,cfl,tfinal),dts)
np.save("./%s/SWE_1D_Err_u_CFL_%1.1f_T_%1.1f.npy"%(folder_name,cfl,tfinal),Err_u)
np.save("./%s/SWE_1D_Err_hu_CFL_%1.1f_T_%1.1f.npy"%(folder_name,cfl,tfinal),Err_hu)

#### Convergence plot

In [ ]:


#plotting
dxs = 1./N.T; dts = cfl*dxs/2 
tick_positions = np.linspace(min(dts), max(dts), 3)
custom_ticks = np.linspace(min(dts), max(dts), 3)
Marks = ['o','s','+','*']
lines = [":","--","-.","-"]
colors = ["red", "blue", "green" ,"darkviolet"]

Sl = [2,3,2,3,2,3]; Coeff = [5e-1,1e0,2e-2,2e-1,5e-4,8e-2]; st = 3; en = 7;

font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 15}
plt.rc('font', **font) 

for i in range(3):
    fig = plt.figure(figsize = (5, 6))
    plt.plot(dts,Err_u[2*i,:],color=colors[0], marker=Marks[0], linestyle=lines[0],linewidth=3, label = "(%d,%d,%d)-h"%(S[2*i],P[2*i],Q[2*i]))
    plt.plot(dts,Err_hu[2*i,:],color=colors[1], marker=Marks[1], linestyle=lines[1],linewidth=3,label = "(%d,%d,%d)-hu"%(S[2*i],P[2*i],Q[2*i]))

    plt.plot(dts,Err_u[2*i+1,:],color=colors[2], marker=Marks[2], linestyle=lines[2],linewidth=3,label = "(%d,%d,%d)-h"%(S[2*i+1],P[2*i+1],Q[2*i+1]))
    plt.plot(dts,Err_hu[2*i+1,:],color=colors[3], marker=Marks[3], linestyle=lines[3],linewidth=3,label = "(%d,%d,%d)-hu"%(S[2*i+1],P[2*i+1],Q[2*i+1]))
    
    plt.plot(dts[st:en],Coeff[2*i]*dts[st:en]**Sl[2*i], "-o", color="gray",linewidth=3,label='slope %d'%(Sl[2*i]))
    plt.plot(dts[st:en], Coeff[2*i+1]*dts[st:en]**Sl[2*i+1], "-s", color="gray",linewidth=3,label='slope %d'%(Sl[2*i+1]))

    plt.xscale("log"); plt.yscale("log"); plt.xlabel('$\Delta t$'); plt.ylabel('Error')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.35),ncol=2)